In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
import shutil
import string
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords #Word Stop
from nltk.tokenize import word_tokenize #Tokenization & Word Stop

In [2]:
to_train = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset_notpreprocessed.csv')
to_train['Combined_Text'] = to_train['Title'] + to_train['Description'] + to_train['Component']
data = to_train.copy()
to_train.head()

,Issue_id,Priority,Title,Description,Component,Combined_Text
0,1,P3,Usability issue with external editors (1GE6IRL),- Setup a project that contains a *.gif resour...,Team,Usability issue with external editors (1GE6IRL...
1,13,P3,CC Discussion: local versioning (1GAT3PL),Would it make sense to have a (the?) projects ...,Team,CC Discussion: local versioning (1GAT3PL)Would...
2,12,P3,Manage/unmanage support and policies (1GALAEG),Currently every new resource added to a shared...,Team,Manage/unmanage support and policies (1GALAEG)...
3,11,P3,API: ISharingManager::load mapping vcm project...,Jean-Michel (08/02/2001 1:38:48 PM); ; The ISh...,Team,API: ISharingManager::load mapping vcm project...
4,10,P3,API - VCM event notification (1G8G6RR),There seems to be a need for a VCM Event notif...,Team,API - VCM event notification (1G8G6RR)There se...


In [3]:
to_get_topics = pd.read_csv('/home/rpierson/PiersonREU/extracted/train_dataset.csv')
to_train['Topic'] = to_get_topics['Topic']


In [4]:
topic_one_data = to_train[to_train['Topic']==1]

In [5]:

extract_dir = '/home/rpierson/PiersonREU/extracted'
topic_one_path = os.path.join(extract_dir, 'topic_one_data.csv')
topic_one_data.to_csv(topic_one_path, index = False)
topic_one_data

,Issue_id,Priority,Title,Description,Component,Combined_Text,Topic
42,44,P3,UI: Term version used for different things (1G...,Some dialogs only show the items that are vers...,Team,UI: Term version used for different things (1G...,1
80,106,P5,Layout odd with Ignore resources pref page (1G...,The ingore resources page has the list box far...,Team,Layout odd with Ignore resources pref page (1G...,1
109,77,P3,DCR: Need Expand All in comparison editor (1GE...,When you compare a Java project with the team ...,Team,DCR: Need Expand All in comparison editor (1GE...,1
115,85,P3,Comparison browser does not save (1GE715H),1) Get a java class from the CVS repository; 2...,Team,Comparison browser does not save (1GE715H)1) G...,1
120,101,P5,Help contexts need to be added for wizards (1G...,merge wizard; new cvs connection; new team str...,Team,Help contexts need to be added for wizards (1G...,1
...,...,...,...,...,...,...,...
68038,229158,P3,[Perspectives] perspectiveExtension schema imp...,I noticed that we can improve the perspectiveE...,UI,[Perspectives] perspectiveExtension schema imp...,1
68043,229182,P3,Wrong product version in eclipse-jee-ganymede-...,I downloaded eclipse-jee-ganymede-M6-win32.zip...,Releng,Wrong product version in eclipse-jee-ganymede-...,1
68045,229204,P3,build fails when running pde.api tools if sdk ...,Should check that sdk includes plugins and fea...,Releng,build fails when running pde.api tools if sdk ...,1
68051,229287,P3,[Intro] Doc rewording,eclipse\plugins\org.eclipse.platform.doc.isv\r...,User Assistance,[Intro] Doc rewordingeclipse\plugins\org.eclip...,1


In [6]:
n = len(topic_one_data)
split_index = int(.8 * n)
train_dataset = topic_one_data.iloc[:split_index]
validation_dataset = topic_one_data.drop(train_dataset.index)
validation_dataset

,Issue_id,Priority,Title,Description,Component,Combined_Text,Topic
59620,171079,P3,[Contributions] enablement state does not upda...,I20070109-1805; ; If I am using a handler with...,UI,[Contributions] enablement state does not upda...,1
59637,171221,P3,Entry on JAR signing in porting guide,The 3.3 porting guide should have an entry on ...,Runtime,Entry on JAR signing in porting guideThe 3.3 p...,1
59647,171319,P3,[KeyBindings] IActionDelegate with key binding...,Build ID: M20060921-0945; ; While trying to us...,UI,[KeyBindings] IActionDelegate with key binding...,1
59652,171361,P3,[ErrorHandling] Platform error dialog not corr...,I20070116-1510 + plug-in export I20070116-1510...,UI,[ErrorHandling] Platform error dialog not corr...,1
59670,171484,P3,javadoc warning in I20070123-1030,./org.eclipse.swt/Eclipse SWT PI/win32/org/ecl...,SWT,javadoc warning in I20070123-1030./org.eclipse...,1
...,...,...,...,...,...,...,...
68038,229158,P3,[Perspectives] perspectiveExtension schema imp...,I noticed that we can improve the perspectiveE...,UI,[Perspectives] perspectiveExtension schema imp...,1
68043,229182,P3,Wrong product version in eclipse-jee-ganymede-...,I downloaded eclipse-jee-ganymede-M6-win32.zip...,Releng,Wrong product version in eclipse-jee-ganymede-...,1
68045,229204,P3,build fails when running pde.api tools if sdk ...,Should check that sdk includes plugins and fea...,Releng,build fails when running pde.api tools if sdk ...,1
68051,229287,P3,[Intro] Doc rewording,eclipse\plugins\org.eclipse.platform.doc.isv\r...,User Assistance,[Intro] Doc rewordingeclipse\plugins\org.eclip...,1


In [7]:
topic_one_train_priority = train_dataset['Priority']
topic_one_train_data = train_dataset['Combined_Text']
topic_one_validation_priority = validation_dataset['Priority']
topic_one_validation_data = validation_dataset['Combined_Text']
def extract_description(topic_one_train_data):
    # Using regular expression to extract text after the first parentheses
    description_train = re.sub(r'^\([^)]+\)\s*', '', topic_one_train_data)
    return description_train.strip()
def extract_description(topic_one_validation_data):
    # Using regular expression to extract text after the first parentheses
    description_validate = re.sub(r'^\([^)]+\)\s*', '', topic_one_validation_data)
    return description_validate.strip()

In [8]:
print(topic_one_train_data.shape)
print(topic_one_validation_data.shape)

(1879,)
(470,)


In [9]:
vectorizer = CountVectorizer(stop_words = 'english', tokenizer = word_tokenize, lowercase = True)
topic_one_train_data = vectorizer.fit_transform(topic_one_train_data)
topic_one_validation_data = vectorizer.transform(topic_one_validation_data)

/home/rpierson/anaconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [10]:
gnb = GaussianNB()
y_pred = gnb.fit(topic_one_train_data.toarray(), topic_one_train_priority).predict(topic_one_validation_data.toarray())

In [20]:
print("Number of correctly labeled points out of a total %d points : %d"
      % (topic_one_validation_data.shape[0], (topic_one_validation_priority == y_pred).sum()))

Number of mislabeled points out of a total 470 points : 394


In [12]:
priority_dispersion = topic_one_data.groupby('Priority').count()

In [13]:
priority_dispersion

,Issue_id,Title,Description,Component,Combined_Text,Topic
Priority,,,,,,
P1,66,66,66,66,66,66
P2,131,131,131,131,131,131
P3,2053,2053,2053,2053,2053,2053
P4,52,52,52,52,52,52
P5,47,47,47,47,47,47


In [16]:
validation_dataset.shape[0]

470

In [17]:
train_dataset.shape[0]

1879